In [1]:
import pandas as pd
import numpy as np
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [2]:
df = pd.read_csv('A1_dataset.csv')

In [3]:
text = df['TEXT'].to_list()
def preprocess_text(text):
    text = lowercase_text(text)
    text = remove_url_html(text)
    text = remove_users(text)
    text = remove_punctuations(text)
    text = remove_whitespaces(text)
    text = tokenization(text)
    text = spelling_correction(text, 'textblob')
    return text
df['preprocessed_text'] = df['TEXT'].apply(preprocess_text)
preprocess_text = df['preprocessed_text'].to_list()

In [4]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [24]:
import pickle
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline

In [25]:
sid = SentimentIntensityAnalyzer()
ls_word_sentiment_vader = []
for word in tqdm(unigram_counts):
    ls_word_sentiment_vader.append((word, sid.polarity_scores(word)['compound']))

with open('ls_word_sentiment_vader.pickle', 'wb') as f:
    pickle.dump(ls_word_sentiment_vader, f)

100%|██████████| 8496/8496 [00:00<00:00, 57774.06it/s]


In [26]:
hf_sentiment_model = pipeline('sentiment-analysis')
ls_word_sentiment_hf = []
for word in tqdm(unigram_counts):
    hf_res = hf_sentiment_model(word)
    score = hf_res[0]['score']
    pos_neg = hf_res[0]['label']
    if pos_neg == 'NEGATIVE':
        score = -score
    ls_word_sentiment_hf.append((word,score))

with open('ls_word_sentiment_vader.pickle', 'wb') as f:
    pickle.dump(ls_word_sentiment_vader, f)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
# bigram_wrapper = Bigrams(preprocess_text)
# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [6]:
# smoother = LMSmoothers(bigrams, vocab_len)
# laplace_smoothed_bigrams = smoother.laplace()

In [7]:
# import pickle
# with open('laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# load the pickle file
import pickle
with open('laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

In [9]:
BigramLM = LanguageModel(laplace_smoothed_bigrams, unigram_counts, 'vader')

In [15]:
sentence_pos = BigramLM.generate_text(['i', 'am'], sentiment=1, length=7)
sentence_neg = BigramLM.generate_text(['i', 'am'], sentiment=-1, length=7)

KeyboardInterrupt: 

In [ ]:
sentence_pos

['i', 'am', 'swamped', 'so', 'much', 'love', 'you', 'nonstop', 'coz']

In [ ]:
sentence_neg

['i', 'am', 'swamped', 'so', 'much', 'love', 'you', 'nonstop', 'coz']

In [ ]:
BigramLM.computePerplexity(sentence_pos), BigramLM.computePerplexity(sentence_neg)

6034315.198668093

In [14]:

sid = SentimentIntensityAnalyzer()
sid.polarity_scores("good")['compound']

0.4404